In [1]:
#AI-Based Gender Inference 
#Director Name and Bio using TMDb 
#25th July 2025 

import os
import pandas as pd
import requests
import json
import time
from tqdm import tqdm
from openai import OpenAI

# --- 🔐 API Keys ---
TMDB_API_KEY = "ef74866feee9084817794614ffbae21d"

#NOTE: PLEASE REPLACE KEY WITH THE ONE PROVIDED OVER EMAIL
OPENAI_API_KEY = ""  

#PLEASE ENDURE movies_random_sample.csv is in Desktop/ra_app
# --- 📁 Load Movie Dataset ---
csv_path = os.path.expanduser("~/Desktop/ra_app/movies_random_sample.csv")
df = pd.read_csv(csv_path)

# --- 📦 Output Containers ---
output_rows = []
failed_imdb_ids = []

# --- 🔄 TMDb Helper Functions ---
def get_tmdb_id(imdb_id):
    try:
        url = f"https://api.themoviedb.org/3/find/{imdb_id}"
        params = {"api_key": TMDB_API_KEY, "external_source": "imdb_id"}
        r = requests.get(url, params=params)
        data = r.json()
        return data['movie_results'][0]['id'] if data['movie_results'] else None
    except:
        return None

def get_movie_title(tmdb_id):
    try:
        url = f"https://api.themoviedb.org/3/movie/{tmdb_id}"
        params = {"api_key": TMDB_API_KEY}
        r = requests.get(url, params=params)
        return r.json().get("title", "Unknown Title")
    except:
        return "Unknown Title"

def get_directors(tmdb_id):
    try:
        url = f"https://api.themoviedb.org/3/movie/{tmdb_id}/credits"
        params = {"api_key": TMDB_API_KEY}
        r = requests.get(url, params=params)
        data = r.json()
        return [(p['id'], p['name']) for p in data.get("crew", []) if p.get("job") == "Director"]
    except:
        return []

def get_bio(person_id):
    try:
        url = f"https://api.themoviedb.org/3/person/{person_id}"
        params = {"api_key": TMDB_API_KEY}
        r = requests.get(url, params=params)
        return r.json().get("biography", "")
    except:
        return ""

# --- 🤖 GPT Inference Function ---
def infer_gender(name, bio, max_retries=3):
    client = OpenAI(api_key=OPENAI_API_KEY)  # ✅ Correctly define client inside the function

    prompt = f"""
Based on the following name and biography of a film director associated with Indian cinema (especially Bollywood), what is the most likely gender of this person?

Name: {name}
Biography: {bio}

Respond in JSON format with two fields:
{{
  "gender": "male" or "female",
  "confidence": a number between 0.0 and 1.0 indicating how confident you are in this classification
}}
"""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                timeout=60
            )
            raw = response.choices[0].message.content.strip()
            result = json.loads(raw)
            return result["gender"].lower(), float(result["confidence"])
        except Exception as e:
            print(f"⚠️ GPT error (attempt {attempt+1}) for {name}: {e}")
            time.sleep(2 + attempt * 2)

    return "unknown", 0.0

# --- 🚀 Main Loop Over Movies ---
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    imdb_id = row['imdb_id']

    tmdb_id = get_tmdb_id(imdb_id)
    if not tmdb_id:
        print(f"❌ TMDb ID not found for IMDb ID {imdb_id}")
        failed_imdb_ids.append({"imdb_id": imdb_id, "reason": "tmdb_id_not_found"})
        continue

    movie_title = get_movie_title(tmdb_id)
    directors = get_directors(tmdb_id)

    if not directors:
        print(f"⚠️ No directors found for: {movie_title} ({imdb_id})")
        failed_imdb_ids.append({"imdb_id": imdb_id, "reason": "no_director_found"})
        continue

    for person_id, director_name in directors:
        bio = get_bio(person_id)
        gender, confidence = infer_gender(director_name, bio)

        output_rows.append({
            "imdb_id": imdb_id,
            "movie_title": movie_title,
            "director_name": director_name,
            "director_bio": bio,
            "inferred_gender": gender,
            "confidence_score": confidence
        })

        print(f"✅ {movie_title} → {director_name} → {gender} ({confidence:.2f})")

        time.sleep(1)  # polite rate limit delay

# --- 💾 Save Output Files ---
output_df = pd.DataFrame(output_rows)
output_path = os.path.expanduser("~/Desktop/ra_app/director_gender_full_output.csv")
output_df.to_csv(output_path, index=False)

if failed_imdb_ids:
    fail_df = pd.DataFrame(failed_imdb_ids)
    fail_path = os.path.expanduser("~/Desktop/ra_app/director_gender_failed_cases.csv")
    fail_df.to_csv(fail_path, index=False)
    print(f"\n⚠️ Logged {len(failed_imdb_ids)} failed cases to: {fail_path}")

print(f"\n✅ Saved final output to: {output_path}")

  0%|                                                   | 0/100 [00:00<?, ?it/s]

✅ My Dear Prime Minister → Rakeysh Omprakash Mehra → male (1.00)


  1%|▍                                          | 1/100 [00:07<13:05,  7.94s/it]

✅ Manikarnika: The Queen of Jhansi → Kangana Ranaut → female (1.00)
✅ Manikarnika: The Queen of Jhansi → Radha Krishna Jagarlamudi → male (1.00)


  2%|▊                                          | 2/100 [00:21<18:29, 11.32s/it]

...

✅ Baby → Neeraj Pandey → male (1.00)


 99%|█████████████████████████████████████████▌| 99/100 [13:37<00:06,  6.47s/it]

⚠️ No directors found for: The Silent Heroes (tt4309284)
✅ Coffee with D → Vishal Mishra → male (1.00)


100%|█████████████████████████████████████████| 100/100 [13:45<00:00,  8.26s/it]


⚠️ Logged 9 failed cases to: /Users/rishabhbijani/Desktop/ra_app/director_gender_failed_cases.csv

✅ Saved final output to: /Users/rishabhbijani/Desktop/ra_app/director_gender_full_output.csv
